# Imports and setup

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, pipeline
import evaluate
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk
import numpy as np

from data_handling import normalize_sentence

device = "cpu"

c:\Users\standakozak\PycharmProjects\Environments\czech_gec\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def load_sentences(filename="data/train/sentence.gold"):
    with open(filename, "r", encoding="UTF-8") as file:
        sentences = file.readlines()
    for sentence in sentences:
        yield normalize_sentence(sentence)


def load_own_dataset(tokenizer, input_file="data/train/sentence.input"):
    with open(input_file, "r", encoding="UTF-8") as file:
        input_sentences = file.readlines()

    input_tokenized = [tokenizer.encode(normalize_sentence(sentence)) for sentence in input_sentences]

    return input_tokenized


In [3]:
model_name = "t5-base"
model_name = "Helsinki-NLP/opus-mt-en-cs"


retrain_tokenizer = False
tokenizer_name = "Helsinki-NLP/opus-mt-en-cs"
#tokenizer_name = "t5-base"


base_tokenizer = AutoTokenizer.from_pretrained(tokenizer_name, return_tensors="pt")
base_seq2seq = AutoModelForSeq2SeqLM.from_pretrained(model_name)


# Tokenizer Training

In [4]:
training_corpus = load_sentences()

if retrain_tokenizer:
    tokenizer = base_tokenizer.train_new_from_iterator(training_corpus, 52000)
else:
    tokenizer = base_tokenizer

base_seq2seq.config.eos_token_id = tokenizer.eos_token_id
base_seq2seq.config.pad_token_id = tokenizer.pad_token_id

In [5]:
sample_pangram = "Ó, náhlý déšť již zvířil prach a čilá laň teď běží s houfcem gazel Ualdewara k exkluzívním úkrytům!"

print("Pangram with base tokenizer: ")
encodings_base = base_tokenizer.encode(sample_pangram)
print(base_tokenizer.decode(encodings_base))
print(f"{len(encodings_base)} tokens")

print("\nFine-tuned tokenizer:")
encodings_used = tokenizer.encode(sample_pangram)
print(tokenizer.decode(encodings_used))
print(f"{len(encodings_used)} tokens")


Pangram with base tokenizer: 
Ó, náhlý déšť již zvířil prach a čilá laň teď běží s houfcem gazel Ualdewara k exkluzívním úkrytům!</s>
59 tokens

Fine-tuned tokenizer:
Ó, náhlý déšť již zvířil prach a čilá laň teď běží s houfcem gazel Ualdewara k exkluzívním úkrytům!</s>
59 tokens


# Seq2Seq Model Training

## GECCC Dataset 

## Own dataset

In [24]:
dataset_dict = load_from_disk("data/czech_news_errors")
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['target', 'source'],
        num_rows: 1483008
    })
    test: Dataset({
        features: ['target', 'source'],
        num_rows: 317788
    })
    dev: Dataset({
        features: ['target', 'source'],
        num_rows: 317788
    })
})

In [7]:
max_length = 128

def preprocess_data(dataset):
    normalized_inputs = [normalize_sentence(sentence) for sentence in dataset["source"]]
    normalized_gold = [normalize_sentence(sentence) for sentence in dataset["target"]]

    model_inputs = tokenizer(normalized_inputs, text_target=normalized_gold, max_length=max_length, truncation=True, padding="max_length")
    return model_inputs

In [8]:
tokenized_datasets = dataset_dict.map(
    preprocess_data,
    batched=True
)

In [25]:
"""
mt_metrics = evaluate.combine(
    ["bleu", "chrf"], force_prefix=True
)


def compute_metrics(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions
    
    predictions = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)

    labels_ids[labels_ids == -100] = tokenizer.pad_token_id
    references = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

    outputs = mt_metrics.compute(predictions=predictions,
                             references=references)

    return outputs"""




metric = evaluate.load("sacrebleu")

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

training_args = Seq2SeqTrainingArguments(
    output_dir='./model',
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=1,
    save_steps=5,
    eval_steps=1,
    max_steps=120,
    eval_strategy="no", #"steps",
    save_strategy="no", #"steps",
    predict_with_generate=True,
    report_to=None,
    metric_for_best_model="chr_f_score",
    load_best_model_at_end=True
)

trainer = Seq2SeqTrainer(
    model=base_seq2seq,
    args=training_args,
    train_dataset=tokenized_datasets["train"].with_format("torch"),
    eval_dataset=tokenized_datasets["dev"].with_format("torch"),
    compute_metrics=compute_metrics
)


max_steps is given, it will override any value given in num_train_epochs


In [26]:
trainer.train()

  1%|          | 1/120 [00:12<24:18, 12.26s/it]

{'loss': 0.5234, 'grad_norm': 2.68664288520813, 'learning_rate': 4.958333333333334e-05, 'epoch': 0.0}


  2%|▏         | 2/120 [00:18<16:36,  8.45s/it]

{'loss': 0.5101, 'grad_norm': 2.5996463298797607, 'learning_rate': 4.9166666666666665e-05, 'epoch': 0.0}


  2%|▎         | 3/120 [00:24<14:43,  7.55s/it]

...

100%|██████████| 120/120 [11:00<00:00,  5.79s/it]

{'loss': 0.4289, 'grad_norm': 2.4045698642730713, 'learning_rate': 0.0, 'epoch': 0.0}


100%|██████████| 120/120 [11:00<00:00,  5.50s/it]

{'train_runtime': 660.3332, 'train_samples_per_second': 0.727, 'train_steps_per_second': 0.182, 'train_loss': 0.4516807119051615, 'epoch': 0.0}


TrainOutput(global_step=120, training_loss=0.4516807119051615, metrics={'train_runtime': 660.3332, 'train_samples_per_second': 0.727, 'train_steps_per_second': 0.182, 'total_flos': 16271215165440.0, 'train_loss': 0.4516807119051615, 'epoch': 0.0003236664940445365})

In [11]:
from transformers import GenerationConfig
gen_config = GenerationConfig(**{'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62508]], 'forced_eos_token_id': 0})
gen_config.save_pretrained("./model")

In [28]:
trainer.save_model("./model")
tokenizer.save_pretrained("./model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[62508]], 'forced_eos_token_id': 0}


('./model\\tokenizer_config.json',
 './model\\special_tokens_map.json',
 './model\\vocab.json',
 './model\\source.spm',
 './model\\target.spm',
 './model\\added_tokens.json')

# Using the model

In [29]:
model = pipeline("translation", "./model")

In [30]:
sample_sentences = dataset_dict["test"].select(list(range(10)))

for datapoint in sample_sentences:
    sentence = datapoint["source"]    
    ground_truth = datapoint["target"]
    model_result = model(sentence)[0]["translation_text"]

    print(f"Input: [{sentence}]\nCorrect: [{ground_truth}]\nModel: [{model_result}]\n")

Input: [miloš zeman tenkrát sice odmítl takové "řeči o levicových inletektulech".]
Correct: [Miloš Zeman tenkrát sice odmítl takové "řeči o levicových intelektuálech".]
Model: [Miloš zeman tenkrát sice odmítl takové "řeči o levičových vstupukulech."]

Input: [A je jedno jestli se aůtoři snažili nebo nesnažili, zkrátka sejim to vbůec nepovedlo.]
Correct: [A je jedno jestli se autoři snažili nebo nesnažili, zkrátka se jim to vůbec nepovedlo.]
Model: [A je jedno jeli se autoři snažili nebo nesnažili, zkrátka sejim to všechno nepovedlo.]

Input: [v prběhu tohoto nde k sobě oba antagonisté najdou cestu, objeví v sobě to dobré a poznají teplo upřmíných lidských citů.]
Correct: [V průběhu tohoto dne k sobě oba antagonisté najdou cestu, objeví v sobě to dobré a poznají teplo upřímných lidských citů.]
Model: [V překladu toto nde k sobě Oba antagonisté najdou cestu, objeví v sobě to dobré a poznají teplo upjatých lidských citů.]

Input: [Mohou to být alkalická mýdla, prací  nebo čizticíí prostře